In [1]:
import os
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import backend as K

W0523 16:01:40.238466 140230193157952 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
BERT_MODEL_URL = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

SEQ_LEN = 512
MAX_EXAMPLE_LEN = 512

BATCH_SIZE = 1
BUFFER_SIZE = 1

In [3]:
from data.load import load_cnn_dailymail


train_dataset, val_dataset, test_dataset, metadata = load_cnn_dailymail()  

Instructions for updating:
Colocations handled automatically by placer.


W0523 16:01:42.197967 140230193157952 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0523 16:01:44.458413 140230193157952 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [24]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [25]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    initialize_vars(sess)
    sess.run(iter.initializer)
    tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = tmp
    tmp2 = model.predict([x0, x1, x2])
    

Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7fce32f2dcf8>>
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.


models

In [13]:
class BertLayer(tf.keras.layers.Layer):
    """
    Custom Keras layer, integrating BERT from tf-hub
    """
    def __init__(self, url, seq_len=512, d_embedding=768, n_fine_tune_layers=None, **kwargs):
        self.url = url
        self.n_fine_tune_layers = n_fine_tune_layers
        self.seq_len = seq_len
        self.d_embedding = d_embedding
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.url,
            trainable=self.trainable,
            name="{}_bert_module".format(self.name)
        )

        trainable_vars = self.bert.variables
        
        if self.trainable:

            # Remove unused layers
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

            # Select how many layers to fine tune
            trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

            # Add to trainable weights
            for var in trainable_vars:
                self._trainable_weights.append(var)

            for var in self.bert.variables:
                if var not in self._trainable_weights:
                    self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        
        input_ids, input_mask, segment_ids = inputs
        
        bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)["sequence_output"]
        return result

    def compute_output_shape(self, input_shape):
        raise ValueError()
        return (input_shape[0], self.seq_len, self.d_embedding)


In [14]:
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0523 12:59:35.651068 140526081922880 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [9]:
config.SEQ_LEN

512